In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from strategy import run_static_strategy, run_dynamic_strategy

In [10]:
# define different stock pairs to test
stock_pairs = [
    # indian pairs
    ("HDFCBANK.NS", "ICICIBANK.NS", "India"),
    ("TCS.NS", "INFY.NS", "India"),
    ("SBIN.NS", "PNB.NS", "India"),

    # us pairs
    ("KO", "PEP", "US"),
    ("MSFT", "AAPL", "US"),
    ("WMT", "TGT", "US"),

    # global pairs
    ("TSCO.L", "SBRY.L", "UK"),
    ("BP", "SHEL", "UK")
]

start_date = "2018-01-01"
end_date = "2025-05-30"
split_date = "2020-03-01"

# prepare folders
os.makedirs("results/plots", exist_ok=True)
results = []

# run both strategies for all pairs
for pick1, pick2, region in stock_pairs:
    print(f"\n🔍 Running pair: {pick1} vs {pick2} ({region})")

    try:
        # download data
        data1 = yf.Ticker(pick1).history(start=start_date, end=end_date)["Close"]
        data2 = yf.Ticker(pick2).history(start=start_date, end=end_date)["Close"]
        df = pd.DataFrame({pick1: data1, pick2: data2}).dropna()

        # dynamic
        try:
            dyn = run_dynamic_strategy(pick1, pick2, df, split_date)
            dyn_ret = dyn["Cumulative Return"].iloc[-1]
            plt.figure(figsize=(10, 4))
            dyn["Cumulative Return"].plot()
            plt.title(f"Dynamic Strategy: {pick1} vs {pick2}")
            plt.grid(True)
            plt.savefig(f"results/plots/{pick1}_{pick2}_dynamic.png")
            plt.close()
        except Exception as e:
            dyn_ret = None
            print(f"⚠️ Dynamic strategy failed: {e}")

        # static
        try:
            stat = run_static_strategy(pick1, pick2, df, split_date)
            stat_ret = stat["Cumulative Return"].iloc[-1]
            plt.figure(figsize=(10, 4))
            stat["Cumulative Return"].plot()
            plt.title(f"Static Strategy: {pick1} vs {pick2}")
            plt.grid(True)
            plt.savefig(f"results/plots/{pick1}_{pick2}_static.png")
            plt.close()
        except Exception as e:
            stat_ret = None
            print(f"⚠️ Static strategy failed: {e}")

        # log the results
        results.append({
            "Pair": f"{pick1} vs {pick2}",
            "Region": region,
            "Static Return": stat_ret,
            "Dynamic Return": dyn_ret
        })

    except Exception as e:
        print(f"❌ Failed for {pick1} vs {pick2}: {e}")

# save the results
summary = pd.DataFrame(results)
summary.to_csv("results/pair_results.csv", index=False)
print("\n✅ Summary saved to results/pair_results.csv")


🔍 Running pair: HDFCBANK.NS vs ICICIBANK.NS (India)

🔍 Running pair: TCS.NS vs INFY.NS (India)

🔍 Running pair: SBIN.NS vs PNB.NS (India)

🔍 Running pair: KO vs PEP (US)

🔍 Running pair: MSFT vs AAPL (US)

🔍 Running pair: WMT vs TGT (US)

🔍 Running pair: TSCO.L vs SBRY.L (UK)

🔍 Running pair: BP vs SHEL (UK)

✅ Summary saved to results/pair_results.csv
